## gpu 확인 ##

```
사용환경
우분투 맥 윈도우
cuda,cudnn 설치
파이썬 3.7~3.10, 아나콘다 가상환경
```



In [ ]:
!nvidia-smi

Sun Nov  5 12:58:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup 라이브러리 불러오기

In [ ]:
!pip install -q peft transformers datasets evaluate
!pip install wandb

In [ ]:
!pip install einops

In [ ]:
# 모델
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)
# 파인튜닝
from peft import (
    LoraConfig,
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
# 파이토치 데이터셋
from datasets import load_dataset
import evaluate
import torch
import os

# model_name_or_path = "klue/roberta-small"
# model_name_or_path = "Hyeonseo/ko_roberta_small_model" #token classification #나쁘지않음
# model_name_or_path = "kyujinpy/KO-Platypus2-7B-ex"
# model_name_or_path = "xlm-roberta-base" #나쁘지않음

model_name_or_path = "EleutherAI/polyglot-ko-1.3b"
# model_name_or_path = "google/mt5-base" # slow tokenizer
# model_name_or_path = "beomi/llama-2-ko-7b"
# model_name_or_path = "microsoft/phi-1_5"






## Dataset


```
로컬 환경 시 해당 경로 사용
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/dataset"

/content/drive/MyDrive/dataset


In [ ]:
# 키워드(인코딩)와 하이라이트 형태
import pandas as pd
df = pd.read_csv("./test7.csv", encoding="UTF-8", index_col=False)
df

,review_id,message,name,sentiment,highlight
0,894957,색감 화사하고 밴딩 쫀쫀합니다 땀 흡수 잘되서 운동 시 잘 쓰고 있어요,만족도,1,색감 화사하고 밴딩 쫀쫀합니다
1,951853,간편하고 가벼운 운동복이 필요해서요바지는 제가 입고 티셔츠는 딸이 입는걸로 넘...,만족도,2,간편하고 가벼운 운동복이 필요해서요바지는 제가 입고 티셔츠는 딸이 입는걸로 넘...
2,954353,좋아요 많이 좋아요잘 쓸게요,만족도,2,많이 좋아요
3,955993,폼으로 마사지하기 귀찮을때 허벅지종아리 편하게풀려고 구매했습니다,만족도,1,NaN
4,992136,기본색상입니다핑크 화이트와 잘어울림,만족도,1,기본색상입니다
...,...,...,...,...,...
4495,1200590,바느질에서부터 차이가 느껴지네요 촘촘하고 견고하고 잘 만들어진 것 같아요 거기다 핏...,마감처리,0,촘하고 견고하고 잘 만들어진 것 같아요 거
4496,1201361,여름에 운동할때 입으려고 주문했는데요시원하고 좋네요주머니 소지품이 빠지지않게 깔끔한...,마감처리,0,니 소지품이 빠지지않게 깔끔한 지퍼와 밴딩 박음까지두벌 구매했는데 잘입을께요
4497,1206510,유명 스포츠 제품처럼 꼼꼼한 마감이니 제품의 질이 조금은 낮습니다,마감처리,0,유명 스포츠 제품처럼 꼼꼼한 마감이니 제품의 질이 조금은 낮습니다
4498,1206800,저는 우비대용으로 쓸 수 있을 것 같아 구매했는데 만족합니다 제가 쓰던 우비는 뻣뻣...,마감처리,0,가 쓰던 우비는 뻣뻣하고 입을때마다 접힌자국이 신경쓰였는데 이 제품도 구겨진자국들이...


## 전처리

In [ ]:
df = df.loc[:,['message','name','sentiment']]

In [ ]:
df.isna().sum()

message      9
name         0
sentiment    0
dtype: int64

In [ ]:
df = df[df['message'].notna() ]
df = df[df['name'] != "만족도"]
df = df.reset_index(drop=True)
df['name'].unique()

array(['착용감', '스타일', '품질', '사이즈', '라인(핏)', '배송', '디자인', '소재', '기능성', '색상',
       '가격', '길이', '신축성', '마감처리'], dtype=object)

In [ ]:
df['label'] = [ df['name'][x] + str(df['sentiment'][x]) for x in range(len(df)) ]
df['message2'] = [ "### review: "+df['message'][x]+" ### keyword: "+df['name'][x]  for x in range(len(df)) ]

In [ ]:
df['sentiment'] = df['sentiment'].replace(2,4).replace(1,3).replace(0,2).replace(-1,1).replace(-2,0)

df['sentiment'].unique()

array([3, 4, 1, 0, 2])

In [ ]:
categories = df['label'].unique()
train_df = pd.DataFrame()
test_df = pd.DataFrame()


for cat in categories:
    category_data = df[df['label'] == cat]
    train_size = int(len(category_data) * 0.8)
    train_data = category_data.sample(n=train_size, replace=True, random_state=42)
    test_data = category_data.drop(train_data.index)

    train_df = pd.concat([train_df, train_data])
    test_df = pd.concat([test_df, test_data])

print(train_df.head())
print(test_df.head())
print(f"트레인 라벨 수: {train_df['label'].unique().__len__()}")
print(f"테스트 라벨 수: {test_df['label'].unique().__len__()}")

                                              message name  sentiment label  \
73        너무두껍지도 않고 발도 편해요짧은스타일찾고 있었는데 너무좋아요 재구매ㄱㄱ합니다  착용감          3  착용감1   
24                              편하고 이쁜데 바로 세일들어가서 ㅠㅠ힝  착용감          3  착용감1   
83  무릎아래가 좀 붙긴 하지만 재단이 잘 되어있어 생각보다 편합니다색깔도 이쁘고 가성비...  착용감          3  착용감1   
31                                         부드러운 착용감 굿  착용감          3  착용감1   
35  벨벳이라 살짝 걱정 했는데면이랑 똑같아요편하고 도톰해서 지금입기딱더 추워지연 기모레...  착용감          3  착용감1   

                                             message2  
73  ### review: 너무두껍지도 않고 발도 편해요짧은스타일찾고 있었는데 너무좋아요...  
24  ### review: 편하고 이쁜데 바로 세일들어가서 ㅠㅠ힝 ### keyword:...  
83  ### review: 무릎아래가 좀 붙긴 하지만 재단이 잘 되어있어 생각보다 편합니...  
31            ### review: 부드러운 착용감 굿 ### keyword: 착용감  
35  ### review: 벨벳이라 살짝 걱정 했는데면이랑 똑같아요편하고 도톰해서 지금입...  
                                         message name  sentiment label  \
0                                   밀리지않고 색상이좋아요  착용감          3  착용감1   
5                                   편하게 입

In [ ]:
train_df = train_df.iloc[:,[4,2]].reset_index(drop=True)
test_df = test_df.iloc[:,[4,2]].reset_index(drop=True)
print(train_df.head())
print(test_df.head())

                                            message2  sentiment
0  ### review: 너무두껍지도 않고 발도 편해요짧은스타일찾고 있었는데 너무좋아요...          3
1  ### review: 편하고 이쁜데 바로 세일들어가서 ㅠㅠ힝 ### keyword:...          3
2  ### review: 무릎아래가 좀 붙긴 하지만 재단이 잘 되어있어 생각보다 편합니...          3
3            ### review: 부드러운 착용감 굿 ### keyword: 착용감          3
4  ### review: 벨벳이라 살짝 걱정 했는데면이랑 똑같아요편하고 도톰해서 지금입...          3
                                            message2  sentiment
0          ### review: 밀리지않고 색상이좋아요 ### keyword: 착용감          3
1          ### review: 편하게 입기 딱 좋아요 ### keyword: 착용감          3
2  ### review: 처음으로 사봤는데 편하고 좋네요 쫌더 살을 빼야할 듯 불끈불끈...          3
3  ### review: 뒷트임이 있어 답답하지 않아 좋아요꼭 운동할 때 아니라도 입기...          3
4  ### review: 소재가 조금 도톰합니다 활동성은 좋습니다 ### keyword...          3


In [ ]:
from datasets import Dataset, DatasetDict
train_d = Dataset.from_pandas(train_df)
test_d = Dataset.from_pandas(test_df)


dataset = DatasetDict({
    "train": train_d,
    "test": test_d
})

In [ ]:
#공백 위치 설정
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom","ployglot")):
    padding_side = "left"
else:
    padding_side = "right"
#토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id  # 여백 토큰과 end of sentence token




# 사이즈 안맞을 경우(polyglot 사용시)
# CUTOFF_LEN = 2048
# def tokenize(prompt, add_eos_token=True):
#     result = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=CUTOFF_LEN,
#         padding=False,
#         return_tensors=None,
#     )
#     if (
#         result["input_ids"][-1] != tokenizer.eos_token_id
#         and len(result["input_ids"]) < CUTOFF_LEN
#         and add_eos_token
#     ):
#         result["input_ids"].append(tokenizer.eos_token_id)
#         result["attention_mask"].append(1)

#     result["labels"] = result["input_ids"].copy()

#     return result


def tokenize_function(examples):
    # max_length=None #=> use the model max length (it's actually the default)
    outputs = tokenizer(examples["message2"], truncation=True)
    # del outputs['token_type_ids']
    return outputs

In [ ]:
tokenized_datasets = dataset.shuffle(seed=42).map(
    tokenize_function,
    batched=True,
    remove_columns=["message2"],)  #message컬럼은 불필요

tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")

Map:   0%|          | 0/3332 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3332
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1922
    })
})

In [ ]:
tokenized_datasets['train'][1000]

{'labels': 3,
 'input_ids': [6,
  6,
  6,
  21924,
  89,
  76,
  11594,
  29,
  3496,
  226,
  11109,
  226,
  14488,
  1591,
  1130,
  3926,
  6501,
  6,
  6,
  9775,
  25997,
  90,
  25010,
  29,
  3844],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

## peft, 모델 설정

In [ ]:
LORA_R = 4 #8 the rank of the update matrices, Lower rank results in smaller update matrices with fewer trainable parameters.
LORA_ALPHA = 16 #  LoRA scaling factor.
LORA_DROPOUT= 0.3  #0.2 The modules (for example, attention blocks) to apply the LoRA update matrices.
# LORA_TARGET_MODULES = ["query","value"]
# LORA_TARGET_MODULES = [ "q_proj", "v_proj" ] #llama
LORA_TARGET_MODULES = ["query_key_value"] #causal



In [ ]:
peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="SEQ_CLS",
)


In [ ]:
#Data collators are objects that will form a batch by using a list of dataset elements as input.
#These elements are of the same type as the elements of train_dataset or eval_dataset.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
id2label = {0: "매우부정", 1: "부정", 2: "중립", 3: "긍정", 4: "매우긍정"}
label2id = {"매우부정": 0, "부정": 1, "중립": 2, "긍정": 3, "매우긍정": 4}


model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=5, id2label=id2label, label2id=label2id,
    pad_token_id=tokenizer.pad_token_id
    # ,ignore_mismatched_sizes=True
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/polyglot-ko-1.3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 796,672 || all params: 1,271,013,376 || trainable%: 0.06268006419469814


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc=accuracy_score(labels , preds)
  return {"accuracy":acc, "f1": f1}

## Training

In [ ]:
!wandb login

wandb: Currently logged in as: jeongdongseon. Use `wandb login --relogin` to force relogin


In [ ]:
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="cllama"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [ ]:
training_args = TrainingArguments(
    output_dir="dongseon/ployglot_test",
    learning_rate= 1e-4, #5e-4
    per_device_train_batch_size=8, #64
    per_device_eval_batch_size=8,  #16
    num_train_epochs= 4 , #120

    optim="adamw_torch",
    # eval_accumulation_steps  = 2, # (더 빠르지만 더 많은 메모리가 필요함).
    # gradient_accumulation_steps  = 128//32 ,  # 역방향/업데이트 단계를 수행하기 전에 기울기를 누적할 업데이트 단계 수
    adam_beta1 = 0.9, #( float, 선택 사항 , 기본값은 0.9) — AdamW 최적화 프로그램 의 beta1  매개변수
    adam_beta2 = 0.999, #( float, 선택 사항 , 기본값은 0.999) — AdamW 최적화 프로그램 의 beta2  매개변수
    adam_epsilon= 1e-8, # ( float, 선택 사항 , 기본값은 1e-8) — AdamW 최적화 프로그램 의 엡실론  매개변수
    # max_grad_norm= 4, # ( float, 선택 사항 , 기본값은 1.0) — 최대 그라데이션 표준(그라디언트 클리핑용).
    fp16 = True, # ( bool, 선택 사항 , 기본값은 False) — 32비트 훈련 대신 fp16 16비트(혼합) 정밀도 훈련을 사용할지
    # label_smoothing_factor = 0.1, # ( float, 선택 사항 , 기본값은 0.0) — 원핫 인코딩 레이블이 각각 0과 1에서 label_smoothing_factor/num_labels,,,label_smoothing_factor + label_smoothing_factor/num_labels.

    weight_decay= 1.5 , #가중치 감소, 커질수록 오버피팅 방지 0.05
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to="wandb",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: jeongdongseon. Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.100900,0.976710
1000,0.696100,1.046606
1500,0.433400,1.145926


TrainOutput(global_step=1668, training_loss=0.702001320086509, metrics={'train_runtime': 225.2044, 'train_samples_per_second': 59.182, 'train_steps_per_second': 7.407, 'total_flos': 1.0222635693441024e+16, 'train_loss': 0.702001320086509, 'epoch': 4.0})

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

## 모델 업로드

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("dongseon/cllama_sentiment", create_pr=1,use_auth_token=True)

adapter_model.safetensors:   0%|          | 0.00/3.19M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dongseon/cllama_sentiment/commit/c7015a916ea200b95b36e24a9d507aae22ddc9db', commit_message='Upload model', commit_description='', oid='c7015a916ea200b95b36e24a9d507aae22ddc9db', pr_url='https://huggingface.co/dongseon/cllama_sentiment/discussions/3', pr_revision='refs/pr/3', pr_num=3)

## 테스트

**로컬 모델 사용하기**

In [ ]:
key = "소재"
# txt = "단추가 짱짱해요 "
# txt = "화면에서는 하늘색인데 진한 파랑색입니다.. 근데 세탁 후 보풀이 생겼어요"
# txt = "배송이 너무너무 늦어요, 그래도 가격은 저렴하게 샀어요"
# txt = "생각보다 더 따뜻하고 프린팅도 귀엽고 색감도 이쁘고 잘산것 같아요"
txt = "흰옷이라서 속이 좀 비치지만 몸에 딱 맞아요"
# txt = "배송이 최악이애요"

inputs = tokenizer(

        f'### review: {txt} ### keyword: {key} '

    , return_tensors="pt")
# del inputs['attention_mask']
del inputs['token_type_ids']

In [ ]:
# from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("dongseon/kobert")
with torch.no_grad():
     # 모델이 있는 디바이스로 입력 데이터 이동
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    # 모델 호출
    logits = model(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
keyword = model.config.id2label[predicted_class_id]
print(keyword)

중립


In [ ]:
# 텐서
print(predicted_class_id)
print(logits)

3
tensor([[-1.6191,  0.8330, -1.1797,  1.8555, -0.5596, -0.9575, -1.5049, -0.7319,
          1.7100, -0.0459, -0.0417, -2.3418, -0.3887, -1.2627]],
       device='cuda:0')


In [ ]:
import torch
import torch.nn.functional as F

# Softmax 함수를 사용하여 확률 분포 계산
probabilities = F.softmax(logits, dim=1)

# 각 클래스(레이블)에 대한 확률 출력
for class_index, probability in enumerate(probabilities[0]):
  if probability.item() > 0.1 :   # 노출조건
    label = model.config.id2label[class_index]
    print(f"Class: {label}, Probability: {probability.item():.4f}")


Class: 기능성, Probability: 0.1188
Class: 디자인, Probability: 0.3303
Class: 색상, Probability: 0.2856


In [ ]:
import numpy as np
lbl = "디자인"

# 모델의 출력에서 '품질' 클래스의 확률
quality_probability = probabilities[0][model.config.label2id[lbl]]

# 각 입력 토큰에 대한 확률 기여도 계산
token_contributions = []
for token_index in range(len(txt.split())):
    # 모델에 입력 토큰 단위로 대체하면서 확률 계산
    replaced_txt = txt.split()
    replaced_txt[token_index] = "[MASK]"
    replaced_txt = " ".join(replaced_txt)

    # 대체된 텍스트를 토큰화하고 입력 데이터 준비
    inputs = tokenizer(replaced_txt, return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    del inputs['token_type_ids']

    # 모델 호출하여 확률 계산
    logits = model(**inputs).logits
    probabilities = F.softmax(logits, dim=1)
    token_quality_probability = probabilities[0][model.config.label2id[lbl]]

    # 현재 토큰의 확률 기여도를 계산하고 리스트에 추가
    token_contribution = quality_probability - token_quality_probability
    token_contributions.append(token_contribution.item())

# 가장 큰 기여도를 가진 토큰과 그 값 찾기
max_contribution_index = np.argmax(token_contributions)
max_contribution_value = token_contributions[max_contribution_index]
max_contributing_token = txt.split()[max_contribution_index]

# 결과 출력
print(f"가장 영향을 많이 주는 토큰: '{max_contributing_token}', 기여도: {max_contribution_value:.4f}")


가장 영향을 많이 주는 토큰: '귀엽고', 기여도: 0.1769


In [ ]:
print(txt.split())
print(token_contributions)

['생각보다', '더', '따뜻하고', '프린팅도', '귀엽고', '색감도', '이쁘고', '잘산것', '같아요']
[0.00721663236618042, 0.011281907558441162, -0.22268304228782654, 0.05458226799964905, 0.17685739696025848, -0.2834058105945587, 0.049906641244888306, 0.0046234130859375, 0.0]


In [ ]:

def find_second_largest_index(nums):
    if len(nums) < 2:
        return None  # 리스트에 두 개 이상의 원소가 없는 경우 None 반환

    largest_index = 0
    second_largest_index = -1

    for i in range(1, len(nums)):
        if nums[i] > nums[largest_index]:
            second_largest_index = largest_index
            largest_index = i
        elif nums[i] != nums[largest_index]:
            if second_largest_index == -1 or nums[i] > nums[second_largest_index]:
                second_largest_index = i

    return second_largest_index

# 두 번째로 큰 숫자의 인덱스 찾기
second_largest_index = find_second_largest_index(token_contributions)
second_largest_index


max_contribution_index
first_negative_index = None
for i, value in enumerate(token_contributions):
  if i > max_contribution_index and value >= 0 :
    first_negative_index = max_contribution_index
  else :
    if i > max_contribution_index and value < 0:
        first_negative_index = i
        break  # 처음으로 마이너스가 되는 곳을 찾으면 반복을 중단합니다.
first_negative_index
index_length = len(txt.split())

print(second_largest_index)
print(max_contribution_index)
print(first_negative_index)
print(index_length)

3
4
5
9


In [ ]:
condition1 = max_contribution_index-second_largest_index

In [ ]:
if second_largest_index < max_contribution_index and token_contributions[second_largest_index] >0 and condition1 < 2:
  start = second_largest_index
else : start = max_contribution_index


if first_negative_index > max_contribution_index :
  if first_negative_index-max_contribution_index == 1:
   end = first_negative_index+1
  else:
   end = first_negative_index
else : end = index_length



for i in range(start,end):
  print(txt.split()[i])

프린팅도
귀엽고
색감도
